##Comentario para el profe :
##Profe al principio tuve unos problemas con coursera ya que no me dejaba descargar los notebooks, esto especificamente para el ejercicio de resolver una cuadratica, pense que era por que no habia enviado la tarea anterior (que es esta), entonces en su momento envie una tarea vacia a esta asignación y aparentemente personas randoms me calificaron la entrega bien, y segun veo no puedo volver a publicar la entrega, entonces solo la subo a github

# I. $LU$ factorization of a square matrix
When we premultply $A$ by lower triangular elemetary matrices $\Lambda_j$ it is trasfrommed in an  upper triangular matrix $U$

$$  \Lambda_{n-1}  \ldots\Lambda_2  \Lambda_1 A  = U$$

$$  A  = (\Lambda_{n-1}  \ldots\Lambda_2  \Lambda_2 )^{-1} U$$

The inverse of a product of matrices is the revesre product of inverses
$$  A  = (\Lambda_1^{-1}  \Lambda_2^{-1} \ldots  \Lambda_{n-1}^{-1}) U$$

and inverse of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_j^{-1} = - \Lambda_j$, so 

$$ A  =  (-\Lambda_1) (-\Lambda_2) \ldots   (-\Lambda_{n-1})  U$$

So the $LU$ column pivot factorization is 
$$  A  = L U$$
with
$$ U = \Lambda_{n-1}  \ldots\Lambda_2  \Lambda_1 A  $$
an upper triangular matrix
$$ L  =  \Lambda_1^{-1}  \Lambda_2^{-1} \ldots  \Lambda_{n-1}^{-1} $$
an lower triangular matrix.

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [24]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [25]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [26]:
a

array([[3.   , 3.   , 3.   , 3.   , 3.   , 3.   ],
       [3.   , 1.875, 1.364, 1.071, 0.882, 0.75 ],
       [3.   , 1.364, 0.882, 0.652, 0.517, 0.429],
       [3.   , 1.071, 0.652, 0.469, 0.366, 0.3  ],
       [3.   , 0.882, 0.517, 0.366, 0.283, 0.231],
       [3.   , 0.75 , 0.429, 0.3  , 0.231, 0.188]])

In [27]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [28]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  2.220e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -4.528e-16  0.000e+00  6.939e-18  8.080e-04  1.902e-03]
 [ 0.000e+00  4.123e-16  0.000e+00 -1.634e-17  0.000e+00 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -1.110e-16  1.110e-16  1.110e-16 -5.551e-17]
 [ 0.000e+00  0.000e+00  3.331e-16 -2.220e-16 -5.551e-17  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -1.110e-16 -1.665e-16  0.000e+00]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [29]:
a1 = a.copy()
a1[1, 1] = 3

In [30]:
np.linalg.matrix_rank(a1)

6

In [31]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in matmul
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in matmul
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [32]:
def comprobacion(A):
  for i in range(0,len(A)):
    if np.linalg.det(A[0:i,0:i]) == 0:
      return False
  return True

comprobacion(a), comprobacion(a1)

(True, False)

### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

# 2. $LU$ factorization column pivoting
When we premultply $A$ by elementary permutation matricex$P_j$( to find a good pivot) and the premultply by lower triangular elemetary matrices $\Lambda_j$  it is trasfrommed in an  upper triangular matrix $U$

$$  \Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1 A  = U$$
$$  A  = (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1} U$$

with

\begin{array}{ll}L  &=  (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1}\\
&= P_1^{-1} \Lambda_1^{-1} P_2^{-1} \Lambda_2^{-1} \ldots  P_{n-1}^{-1}\\
 &= P_1^{t} (-\Lambda_1) P_2^{t} (-\Lambda_2) \ldots  P_{n-1}^{t} (-\Lambda_{n-1})
\end{array}



Because the inverse a of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_i^{-1} = - \Lambda_i$ and the inverse of a Permutation Matrix (in particular an elementary permutation matrix)  is its transpose [math.stackexchange](
        https://math.stackexchange.com/questions/98549/the-transpose-of-a-permutation-matrix-is-its-inverse#:~:text=Taking%20the%20transpose%20of%20P,Pt%3DP%E2%88%921.)  $P_i^{-1}=P_i^{t}$, so


If we use 
$$  \hat{L}  = (\Lambda_{n-1}  \ldots \Lambda_2\Lambda_1)^{-1}$$
The inverse of a product of matrices is the revesre product of inverses 
$$ \hat{L} = \Lambda_1^{-1}\Lambda_2^{-1} \ldots  \Lambda_{n-1}^{-1} $$
the inverse a of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_i^{-1} = - \Lambda_i$
$$ \hat{L} = (-\Lambda_1)  (-\Lambda_2) \ldots  (-\Lambda_{n-1}) $$

Then
$$    A_\pi  = \hat{L} U$$
with $A_\pi$ equal to $A$ with rows permutated acoording to some permutation $\pi$

Equiavlently 
$$   A  = (\hat{L}U)_{\pi'}$$
 $(\hat{L}U)_{\pi'}$ equal to $\hat{L}U$ with rows permutated acoording to some permutation $\pi'$

In [33]:
#Primer punto del test 2
def swapper(matriz,filaA,filaB):
  #Esta función me intercambia los elementos  de la fila a con los de la fila b
  matriz[[filaA,filaB]] = matriz[[filaB,filaA]]

def diy_lu_pivote(a):

    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    P = np.eye(N) # Cambio
    for j in range(N-1):
        lam = np.eye(N)
        #inicio Cambio
        piv = j;
        for k in range (j,N-j-1):
            if u[piv,j] <= np.abs(u[k,j]):
                piv = k;
          
        oP = np.eye(N)
        swapper(oP,piv,j)

        u = oP @ u
        P = oP @ P

        #Fin Cambio
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u, P

In [34]:
N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

L, u, P = diy_lu_pivote(a)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a\n",a, "\n")
print("P\n", P, "\n")
print("P@L@u\n", P@L@u, "\n")
print("a\n",a, "\n")

L
 [[ 1.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  1.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  4.848e-01  1.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  1.905e-01  5.807e-01  1.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  2.133e+00 -1.496e+01  3.297e+02  1.000e+00  0.000e+00]
 [ 1.000e+00 -1.333e-01 -5.343e-01 -1.766e+00  2.747e-04  1.000e+00]] 

u
 [[ 3.000e+00  8.824e-01  5.172e-01  3.659e-01  2.830e-01  2.308e-01]
 [ 0.000e+00  9.926e-01  8.464e-01  7.056e-01  5.993e-01  5.192e-01]
 [ 0.000e+00  0.000e+00 -4.526e-02 -5.578e-02 -5.636e-02 -5.395e-02]
 [ 0.000e+00  0.000e+00 -3.469e-18  8.932e-04  1.409e-03  1.659e-03]
 [ 0.000e+00  0.000e+00  1.144e-15  0.000e+00  1.307e-01  3.076e-01]
 [ 0.000e+00  0.000e+00 -9.912e-18  0.000e+00  0.000e+00 -1.585e-05]] 

L@u
 [[3.    0.882 0.517 0.366 0.283 0.231]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]

In [35]:
a1 = a.copy()
a1[1, 1] = 3

L, u , P = diy_lu_pivote(a1)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a\n",a, "\n")
print("P\n", P, "\n")
print("P@L@u\n", P@L@u, "\n")
print("a\n",a, "\n")

L
 [[ 1.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  1.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  2.273e-01  1.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  8.929e-02  3.436e-01  1.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  1.000e+00  9.472e+00 -2.172e+02  1.000e+00  0.000e+00]
 [ 1.000e+00 -6.250e-02 -2.071e-01 -1.278e+00 -2.652e-04  1.000e+00]] 

u
 [[ 3.000e+00  8.824e-01  5.172e-01  3.659e-01  2.830e-01  2.308e-01]
 [ 0.000e+00  2.118e+00  8.464e-01  7.056e-01  5.993e-01  5.192e-01]
 [ 0.000e+00  0.000e+00  1.727e-01  1.260e-01  9.801e-02  7.980e-02]
 [ 0.000e+00  0.000e+00 -6.939e-18 -3.386e-03 -4.356e-03 -4.549e-03]
 [ 0.000e+00  0.000e+00 -1.507e-15  0.000e+00  2.430e-01  5.061e-01]
 [ 0.000e+00  0.000e+00 -9.268e-18  0.000e+00  0.000e+00  2.567e-05]] 

L@u
 [[3.    0.882 0.517 0.366 0.283 0.231]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]

In [36]:
a2 = np.array([[4,3,1], [5,7,0], [9,9,3], [8,2,4]])

L, u, P = diy_lu_pivote(a2)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a\n",a, "\n")
print("P\n", P, "\n")
print("P@L@u\n", P@L@u, "\n")
print("a\n",a, "\n")

L
 [[ 1.     0.     0.     0.   ]
 [ 0.556  1.     0.     0.   ]
 [ 0.444 -0.5    1.     0.   ]
 [ 0.889 -3.     3.143  1.   ]] 

u
 [[ 9.     9.     3.   ]
 [ 0.     2.    -1.667]
 [ 0.     0.    -1.167]
 [ 0.     0.     0.   ]] 

L@u
 [[9. 9. 3.]
 [5. 7. 0.]
 [4. 3. 1.]
 [8. 2. 4.]] 

a
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]] 

P
 [[0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]] 

P@L@u
 [[4. 3. 1.]
 [5. 7. 0.]
 [9. 9. 3.]
 [8. 2. 4.]] 

a
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]] 



# 3. $LU$ factorization column pivoting and reconstruction
When we premultply $A$ by elementary permutation matricex$P_j$( to find a good pivot) and the premultply by lower triangular elemetary matrices $\Lambda_j$  it is trasfrommed in an  upper triangular matrix $U$

$$  \Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1 A  = U$$
$$  A  = (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1} U$$
$$  A  = (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1} U$$

The inverse of a product of matrices is the revesre product of inverses 
$$  A  = ( P_1^{-1} \Lambda_1^{-1} P_2^{-1} \Lambda_2^{-1} \ldots  P_{n-1}^{-1}\Lambda_{n-1}^{-1} ) U$$

So the $LU$ column pivot factorization is 
$$  A  = L U$$
with
$$ U = \Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1 A  $$



$$ L  = P_1^{-1} \Lambda_1^{-1} P_2^{-1} \Lambda_2^{-1} \ldots  P_{n-1}^{-1}\Lambda_{n-1}^{-1} $$

the inverse a of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_i^{-1} = - \Lambda_i$ and the inverse of a Permutation Matrix (in particular an elementary permutation matrix)  is its transpose [math.stackexchange](
        https://math.stackexchange.com/questions/98549/the-transpose-of-a-permutation-matrix-is-its-inverse#:~:text=Taking%20the%20transpose%20of%20P,Pt%3DP%E2%88%921.)  $P_i^{-1}=P_i^{t}$, so

$$ L  =  P_1^{t} (-\Lambda_1) P_2^{t} (-\Lambda_2) \ldots  P_{n-1}^{t} (-\Lambda_{n-1}) $$


In [37]:
def reconstructor(L,U,P):
    return P@L@U
#Creo que estes no el formato que el profe quiere X.X

In [38]:
N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

L, u , P = diy_lu_pivote(a1)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a\n",a, "\n")
print("Matriz a reconstruida : ", reconstructor(L,u,P))

L
 [[ 1.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  1.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  2.273e-01  1.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  8.929e-02  3.436e-01  1.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  1.000e+00  9.472e+00 -2.172e+02  1.000e+00  0.000e+00]
 [ 1.000e+00 -6.250e-02 -2.071e-01 -1.278e+00 -2.652e-04  1.000e+00]] 

u
 [[ 3.000e+00  8.824e-01  5.172e-01  3.659e-01  2.830e-01  2.308e-01]
 [ 0.000e+00  2.118e+00  8.464e-01  7.056e-01  5.993e-01  5.192e-01]
 [ 0.000e+00  0.000e+00  1.727e-01  1.260e-01  9.801e-02  7.980e-02]
 [ 0.000e+00  0.000e+00 -6.939e-18 -3.386e-03 -4.356e-03 -4.549e-03]
 [ 0.000e+00  0.000e+00 -1.507e-15  0.000e+00  2.430e-01  5.061e-01]
 [ 0.000e+00  0.000e+00 -9.268e-18  0.000e+00  0.000e+00  2.567e-05]] 

L@u
 [[3.    0.882 0.517 0.366 0.283 0.231]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]

In [39]:
a1 = a.copy()
a1[1, 1] = 3

L, u , P = diy_lu_pivote(a1)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a2\n",a1, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L@u - a1\n",L@u - a1, "\n")
print("Matriz a1 reconstruida : ", reconstructor(L,u,P))

L
 [[ 1.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  1.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  2.273e-01  1.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  8.929e-02  3.436e-01  1.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  1.000e+00  9.472e+00 -2.172e+02  1.000e+00  0.000e+00]
 [ 1.000e+00 -6.250e-02 -2.071e-01 -1.278e+00 -2.652e-04  1.000e+00]] 

u
 [[ 3.000e+00  8.824e-01  5.172e-01  3.659e-01  2.830e-01  2.308e-01]
 [ 0.000e+00  2.118e+00  8.464e-01  7.056e-01  5.993e-01  5.192e-01]
 [ 0.000e+00  0.000e+00  1.727e-01  1.260e-01  9.801e-02  7.980e-02]
 [ 0.000e+00  0.000e+00 -6.939e-18 -3.386e-03 -4.356e-03 -4.549e-03]
 [ 0.000e+00  0.000e+00 -1.507e-15  0.000e+00  2.430e-01  5.061e-01]
 [ 0.000e+00  0.000e+00 -9.268e-18  0.000e+00  0.000e+00  2.567e-05]] 

L@u
 [[3.    0.882 0.517 0.366 0.283 0.231]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]

In [40]:
a2 = np.array([[4,3,1], [5,7,0], [9,9,3], [8,2,4]])

L, u , P = diy_lu_pivote(a1)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a2\n",a2, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
#print("L@u - a2\n",L@u - a2, "\n")
print("Matriz a2 reconstruida : ", reconstructor(L,u,P))

L
 [[ 1.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  1.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  2.273e-01  1.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  8.929e-02  3.436e-01  1.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  1.000e+00  9.472e+00 -2.172e+02  1.000e+00  0.000e+00]
 [ 1.000e+00 -6.250e-02 -2.071e-01 -1.278e+00 -2.652e-04  1.000e+00]] 

u
 [[ 3.000e+00  8.824e-01  5.172e-01  3.659e-01  2.830e-01  2.308e-01]
 [ 0.000e+00  2.118e+00  8.464e-01  7.056e-01  5.993e-01  5.192e-01]
 [ 0.000e+00  0.000e+00  1.727e-01  1.260e-01  9.801e-02  7.980e-02]
 [ 0.000e+00  0.000e+00 -6.939e-18 -3.386e-03 -4.356e-03 -4.549e-03]
 [ 0.000e+00  0.000e+00 -1.507e-15  0.000e+00  2.430e-01  5.061e-01]
 [ 0.000e+00  0.000e+00 -9.268e-18  0.000e+00  0.000e+00  2.567e-05]] 

L@u
 [[3.    0.882 0.517 0.366 0.283 0.231]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]